In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import datetime as dt
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
data_path = r'../data/clean_df.csv.gz'
df = pd.read_csv(data_path).drop('Unnamed: 0', axis=1)
df.head()

BadGzipFile: CRC check failed 0xe51fccdd != 0x66f59042

In [ ]:
df.info()

In [ ]:
df['CRASH TIME'] = pd.to_datetime(df['CRASH TIME'])
df['CRASH DATE'] = pd.to_datetime(df['CRASH DATE'])

In [ ]:
df['BOROUGH'].hist()

A lot more accidents in Brooklyn and Queens than in Manhattan. Unexpected. Let's examine each of those boroughs and determine if there are specific "danger areas."

In [ ]:
brooklyn = df[df['BOROUGH'] == 'BROOKLYN']
brooklyn.loc[:,'ON STREET NAME'] = brooklyn['ON STREET NAME'].apply(lambda x: str(x).rstrip())

In [ ]:
brooklyn.head()

##### Create a table that examines the number of accidents on a street, divided by the max Euclidean distance between the furthest accidents. You can do this by creating a Euclidean distance for each accident from (0,0) and taking the max for that column for each street name.

### Accidents pivot table

In [ ]:
lat_range = brooklyn.groupby('ON STREET NAME')['LATITUDE'].max() - brooklyn.groupby(('ON STREET NAME'))['LATITUDE'].min()
lon_range = brooklyn.groupby('ON STREET NAME')['LONGITUDE'].max() - brooklyn.groupby(('ON STREET NAME'))['LONGITUDE'].min()
euclidean_range = np.sqrt(lat_range ** 2 + lon_range ** 2)

In [ ]:
brooklyn.loc[:,'RANGE'] = brooklyn['ON STREET NAME'].apply(lambda x: euclidean_range[x])

In [ ]:
euclidean_range_no_nans = euclidean_range[euclidean_range.index != 'nan']
euclidean_range_no_nans.sort_values(ascending=False)

In [ ]:
casualties = brooklyn.groupby('ON STREET NAME')['TOTAL PEDESTRIAN CASUALTIES'].sum()
casualties_no_nans = casualties[casualties.index != 'nan']
casualties_no_nans.sort_values(ascending=False)

In [ ]:
euclidean_range.corr(casualties_no_nans)

In [ ]:
_ = plt.figure(figsize=(8,8))
_ = plt.scatter(euclidean_range_no_nans, casualties_no_nans)
_ = plt.xlabel('Euclidean range', fontsize=12)
_ = plt.ylabel('Pedestrian casualties', fontsize=12)
_ = plt.title('Street length vs. casualties', fontsize=16)

The pedestrian casualty count does not completely correlate to the Euclidean length of the street. For example, the highest pedestrian casualty count is on Bedford Ave, but the longest street by Euclidean measurement is Atlantic Ave.

In [ ]:
street_count = brooklyn.groupby('ON STREET NAME')['COLLISION_ID'].count()
street_count

In [ ]:
streets = pd.DataFrame({'# ACCIDENTS':street_count, 'MAP RANGE':euclidean_range, 'CASUALTIES':casualties},
                              index=euclidean_range.index)
streets.sort_values('CASUALTIES', ascending=False).head(10)

In [ ]:
streets = streets.drop(streets.sort_values('CASUALTIES', ascending=False).index[0], axis=0)
streets.sort_values('CASUALTIES', ascending=False).head(10)

In [ ]:
_ = streets.hist(figsize=(6,6))

In [ ]:
_ = plt.plot(streets['# ACCIDENTS'], streets['CASUALTIES'], marker='.', linestyle='none')

In [ ]:
streets.corr()

Atlantic Ave is clearly an outlier with regard to number of accidents and number of casualties (since those are highly correlated, this is not a surprise). Is it related to the fact that the street is longer than most others?

In [ ]:
map_head = streets.sort_values('MAP RANGE', ascending=False).head(50)
map_head.head()

In [ ]:
_ = plt.plot(map_head['MAP RANGE'], map_head['# ACCIDENTS'], marker='.', linestyle='none')
_ = plt.title('Correlation: {}'.format(map_head['MAP RANGE'].corr(map_head['# ACCIDENTS'])))

While a single accident may occur on a street, we are more interested in patterns. As such, we should consider dropping records where `# ACCIDENTS` equals 1.

In [ ]:
len(streets[streets['# ACCIDENTS'] == 1])

In [ ]:
more_accidents = streets[streets['# ACCIDENTS'] >= 10]
more_accidents.sort_values('# ACCIDENTS', ascending=True).head(10)

In [ ]:
more_accidents.corr()

##### Add daytime and weekend data to `more_accidents`

In [ ]:
daytime_weekend = df.groupby('ON STREET NAME')[['WEEKEND', 'DURING DAYTIME']].sum()
dt_wk_streets = more_accidents.join(daytime_weekend, how='right')

In [ ]:
dt_wk_streets.head()

Where are the accidents that don't have street labels? Maybe they're on major highways.

In [ ]:
no_streets = df[(df['ON STREET NAME'].isnull() == True) & (df['BOROUGH'] == 'BROOKLYN')]

_ = plt.figure(figsize=(14,14))
_ = plt.plot(no_streets['LONGITUDE'], no_streets['LATITUDE'], marker='.', linestyle='none', alpha=0.03)
_ = plt.xlim(-74.05, -73.85)
_ = plt.ylim(40.55, 40.75)
plt.show()

### Hourly plots

##### Hourly data by borough

In [ ]:
boroughs = df['BOROUGH'].dropna().unique()
plt.figure(figsize=(14,10))
for borough in boroughs:
    hours = df.loc[(df['BOROUGH'] == borough) == True].groupby(df['CRASH TIME'].dt.hour)\
    ['TOTAL PEDESTRIAN CASUALTIES'].sum()
    
    _ = plt.plot(range(0,24), hours, linestyle='-', linewidth=2.5)
    _ = plt.xticks(np.arange(0,24,1))
_ = plt.grid()
_ = plt.legend(boroughs)

1. Manhattan and the Bronx have a spike at 8 AM. Brooklyn hits that same high, but doesn't taper off afterwards. Manhattan doesn't spike at 8 AM at all.
2. All dip at 3 PM and spike at 4 PM then taper into the evenings.

##### Hourly data by day

In [ ]:
days = df['WEEKDAY'].unique()
plt.figure(figsize=(14,10))
for day in days:
    hours = df.loc[(df['WEEKDAY'] == day) == True].groupby(df['CRASH TIME'].dt.hour)\
    ['TOTAL PEDESTRIAN CASUALTIES'].sum()
    
    _ = plt.plot(range(0,24), hours, linestyle='-', linewidth=2.5)
    _ = plt.xticks(np.arange(0,24,1))
_ = plt.grid()
_ = plt.legend(days)
_ = plt.savefig(r'../Image resources/Weekdays.png')

1. Friday doesn't have an early morning spike.
2. There's always a dip at 3 PM.
3. The afternoon rush-hour spike is larger than the morning rush-hour spike.

In [ ]:
df['CRASH YEAR'] = df['CRASH DATE'].dt.year
year_counts = df.groupby('CRASH YEAR')['TOTAL PEDESTRIAN CASUALTIES'].sum()
_ = plt.figure(figsize=(14,10))
_ = plt.bar(x=year_counts.index, height=year_counts)
_ = plt.xticks(range(2012,2021))
_ = plt.title('Yearly pedestrian casualties', fontsize=16)
_ = plt.savefig(r'../Image resources/Years.png')

2013, 2017 and 2018 seem unusually high compared to 2014-2016. 2020 is understandably low, both because of COVID and because the year is not over yet.

In [ ]:
seasons = df['SEASON'].unique()
plt.figure(figsize=(14,10))
for season in seasons:
    hours = df.loc[(df['SEASON'] == season) == True].groupby(df['CRASH TIME'].dt.hour)\
    ['TOTAL PEDESTRIAN CASUALTIES'].sum()
    

    _ = plt.plot(range(0,24), hours, linestyle='-', linewidth=2.5)
    _ = plt.xticks(np.arange(0,24,1))
_ = plt.grid()
_ = plt.legend(seasons)
_ = plt.title('Seasonal pedestrian casualties by hour', fontsize=16)
_ = plt.savefig(r'../Image resources/Seasons.png')

1. The data follows the same trend in each season.
2. There are more fall accidents than winter accidents in the morning.
3. There are the fewest summer accidents in the morning but the most in the evening.
4. Winter's accidents are second-highest in the morning but extemely low in the evening.

In [ ]:
_ = plt.hist(df['LONGITUDE'])

In [ ]:
_ = plt.hist(df['LATITUDE'])

In [ ]:
boroughs = ['MANHATTAN','BRONX','BROOKLYN','STATEN ISLAND','QUEENS']
_ = plt.plot(df['LONGITUDE'], df['LATITUDE'], 'k.', alpha=0.01)
for borough in boroughs:
    df_borough = df[df['BOROUGH'] == borough]
    _ = plt.plot(df_borough['LONGITUDE'].mean(), df_borough['LATITUDE'].mean(), 'or')

In [ ]:
for borough in boroughs:
    df_borough = df['BOROUGH'] == borough
    borough_lat_mean = df.loc[df_borough,'LATITUDE'].mean()
    borough_lon_mean = df.loc[df_borough,'LONGITUDE'].mean()
    lon_distance = (df.loc[df_borough,'LONGITUDE'] - borough_lon_mean) ** 2
    lat_distance = (df.loc[df_borough,'LATITUDE'] - borough_lat_mean) ** 2
    df.loc[df_borough,'DISTANCE FROM MEAN'] = np.sqrt(lon_distance + lat_distance)

In [ ]:
no_borough = df[df['BOROUGH'].isnull() == True]
_ = plt.figure(figsize=(40,40))
_ = sns.scatterplot(x='LONGITUDE',
                    y='LATITUDE',
                    data=no_borough,
                    hue=no_borough['CRASH DATE'].dt.year,
                    cmap='hus1',
                    alpha=0.1)

Null data seems to focus on the borders and through-streets. Are those highways?

In [ ]:
no_borough['ON STREET NAME'].value_counts()[lambda x:x>1000]

In [ ]:
top_counts = df.groupby('DAYS FROM NEW YEARS')['DAYS FROM NEW YEARS'].count()
_ = plt.scatter(top_counts.index, top_counts, marker='.')
_ = plt.title('Number of accidents by day')
_ = plt.tick_params(
    axis='x',          
    which='both',      
    bottom=False,   
    top=False,         
    labelbottom=False) 
_ = plt.show()

In [ ]:
top_counts[top_counts < 3000]

In [ ]:
top_counts.describe()

There is a particular dip around Christmas and New Years. This is to be expected. It is surprising that New Years is relatively so high.

In [ ]:
with pd.option_context('display.max_rows', None):
    print(df['CONTRIBUTING FACTOR VEHICLE 1'].value_counts().sort_values(ascending=False))

It may be worth combining these into a much more limited set of features, e.g. _distracted driver, DUI,_ etc.

### Term distribution

We are count-vectorizing `ON STREET NAME` and `CROSS STREET NAME`. However, a problem arises: When writing down street names, a shorthand is often used, such as "St" for "Street." This will cause a significant amount of confusion in the model. We must determine if this is occurring, and if so, is it on a significant scale.

In [ ]:
df['ON STREET NAME'].fillna('', inplace=True)
count = CountVectorizer(min_df=30, max_df=0.9)
count_vec = count.fit_transform(df['ON STREET NAME'])
count_df = pd.DataFrame.sparse.from_spmatrix(count_vec)

In [ ]:
alt_terms = [['st','street'],
             ['ave','avenue'],
             ['blvd','boulevard'],
             ['rd','road'],
             ['dr','drive'],
             ['crescent','cresc'],
             ['place','pl'],
             ['court','ct'],
             ['terrace','ter'],
             ['highway','hwy'],
             ['parkway','pkwy'],
             ['expressway','expwy'],
             ['junction','jct'],
             ['lane','ln'],
             ['square','sqr'],
             ['extension','ext'],
             ['freeway','frwy'],
             ['plaza','plz'],
             ['tunnel','tunl'],
             ['turnpike','tpke']
            ]
for term in alt_terms:
    try:
        print(term[0], count.vocabulary_[term[0]],
             term[1], count.vocabulary_[term[1]])
    except KeyError:
        continue

It appears to be a major issue. We will have to clean this.

In [ ]:
alt_terms = ['STREET',
             'AVENUE',
             'BOULEVARD',
             'ROAD',
             'DRIVE',
             'PARKWAY',
             'EXPRESSWAY'
            ]
clean_terms = ['ST',
               'AVE',
               'BLVD',
               'RD',
               'DR',
               'PKWY',
               'EXPWY'
              ]
def alt_terms_clean(string):
    for alt, clean in zip(alt_terms, clean_terms):
        string = string.strip().replace(alt, clean)
    return string

df['ON STREET NAME'] = df['ON STREET NAME'].map(alt_terms_clean)

In [ ]:
%%timeit
df['ON STREET NAME'].map(alt_terms_clean)

In [ ]:
%%timeit
df['ON STREET NAME'].map(alt_terms_clean)

In [ ]:
df['CROSS STREET NAME'].fillna('', inplace=True)
df['CROSS STREET NAME'].map(alt_terms_clean)

In [ ]:
df['YEAR-MONTH'] = df['CRASH DATE'].dt.year.astype('str')+'-'+df['CRASH DATE'].dt.strftime('%m').astype('str')
df = df.sort_values(by='YEAR-MONTH', ascending=True)

In [ ]:
_ = plt.figure(figsize=(10,10))
_ = df.groupby('YEAR-MONTH')['TOTAL PEDESTRIAN CASUALTIES'].sum().plot()
_ = plt.title('CASUALTIES BY MONTH', fontsize=16)
_ = plt.ylabel('PEDESTRIAN CASUALTIES')
_ = plt.savefig(r'../Image resources/Casualties by month.png')

In [ ]:
from statsmodels.tsa.stattools import adfuller
time_series = df.groupby('YEAR-MONTH')['TOTAL PEDESTRIAN CASUALTIES'].sum()
adf = adfuller(time_series)

p_value = adf[0]
critical_value = adf[4]['5%']
reject_hypothesis = 'do not' if critical_value < p_value else ''

print(f'The p-value of the Dickey Fuller test is {p_value}.')
print(f'The critical value is {critical_value}.')
print(f'We {reject_hypothesis} reject the null hypothesis that the time series is non-stationary.')

In [3]:
from statsmodels.graphics.tsaplots import plot_acf
time_chg = time_series.diff()

plot_acf(time_chg)

NameError: name 'time_series' is not defined

In [ ]:
df.to_csv(data_path)